# メインコード
サーバサイドはこれを実行すればOK．

In [ ]:
from bottle import get, post, run, HTTPResponse
import requests
import csv
import json

## 便利な関数群
各API部から共通で呼び出される機能をここにまとめています

In [ ]:
def csvreader(fileaddr = '../data/jk.csv'):
    csvlist = []
    with open(fileaddr, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            csvlist.append((row[0],row[1]))
    
    return csvlist

## Exchange
文字列を変換して返すAPI群

### exhangejk
jk用語を共通語に変換する

getメソッド
パラメータ
message : 変換元のメッセージ

返り値
appliction/json形式
status  : ステータスコード
message : 変換したメッセージ

In [ ]:
@get('/exchangejk/<message>')
def hello2(message):
    #message = 'そマ? テンサゲ～'
    exchange_tapple = csvreader('../data/jk.csv')
    for before, after in exchange_tapple:
        message = message.replace(before, after)
    
    body = {"status": "200", "message": message}
    r = HTTPResponse(status=200, body=body)
    r.set_header("Content-Type", "application/json")
    return r

### exhangeit
it技術者が使う用語を共通語に変換する

getメソッド
パラメータ
message : 変換元のメッセージ

返り値
appliction/json形式
status  : ステータスコード
message : 変換したメッセージ

In [ ]:
@get('/exchangeit/<message>')
def hello2(message):
    #message = 'オンスケでコミットできて幸せ'
    exchange_tapple = csvreader('../data/it.csv')
    for before, after in exchange_tapple:
        message = message.replace(before, after)
    
    body = {"status": "200", "message": message}
    r = HTTPResponse(status=200, body=body)
    r.set_header("Content-Type", "application/json")
    return r

## 実行
以上のAPIをポート8080番で実行する

In [ ]:
run(host='0.0.0.0', port=8080, debug=True)